In [2]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

from scipy import stats
from scipy.interpolate import interp1d
from scipy.optimize import fsolve
from scipy.optimize import minimize

In [3]:
data = fpa_data = pd.read_stata('chile.dta')

In [4]:
olsdata = data.copy()
olsdata[['routput','totlab','rcapstock']] = np.log(olsdata[['routput','totlab','rcapstock']])

olsmodel = smf.ols('routput ~totlab + rcapstock',olsdata)
olsresult = olsmodel.fit()
olsresult.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                routput   R-squared:                       0.769
Model:                            OLS   Adj. R-squared:                  0.769
Method:                 Least Squares   F-statistic:                 3.452e+04
Date:                Wed, 08 May 2019   Prob (F-statistic):               0.00
Time:                        11:23:30   Log-Likelihood:                -23690.
No. Observations:               20730   AIC:                         4.739e+04
Df Residuals:                   20727   BIC:                         4.741e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.1804      0.024    214.199      0.000       5.133       5.228
totlab         0.8194      0.008    102.168      0.000       0.804       0.835
rcapstock      0.3317      0.004     91.134      0.000       0.325       0.339
==============================================================================
Omnibus:                     1840.946   Durbin-Watson:                   0.531
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3645.736
Skew:                           0.597   Prob(JB):                         0.00
Kurtosis:                       4.672   Cond. No.                         47.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [44]:
fedata = olsdata.copy()

#apply the within transformation
for var in ['routput','totlab','rcapstock']:
    fedata[var+'fe'] = fedata[var]
    for fe in ['year']:
        dum = pd.get_dummies(fedata[fe])
        means = np.tile(fedata[var],len(dum.columns)).reshape(fedata.shape[0],len(dum.columns))
        means = np.array(means*dum)
        means = means.sum(axis=0)/dum.sum(axis=0)
        
        means = np.tile(means,fedata.shape[0]).reshape(fedata.shape[0],len(dum.columns))
        means = (means*dum).sum(axis=1)
        fedata[var+'fe'] = fedata[var+'fe'] - means

In [50]:
femodel = smf.ols('routputfe ~totlabfe + rcapstockfe -1',fedata)
feresult = femodel.fit()
feresult.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              routputfe   R-squared:                       0.769
Model:                            OLS   Adj. R-squared:                  0.769
Method:                 Least Squares   F-statistic:                 3.443e+04
Date:                Wed, 08 May 2019   Prob (F-statistic):               0.00
Time:                        12:24:52   Log-Likelihood:                -23722.
No. Observations:               20730   AIC:                         4.745e+04
Df Residuals:                   20728   BIC:                         4.746e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
totlabfe        0.8206      0.008    102.206      0.000       0.805       0.836
rcapstockfe     0.3316      0.004     91.017      0.000       0.325       0.339
==============================================================================
Omnibus:                     1822.259   Durbin-Watson:                   0.532
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3590.014
Skew:                           0.594   Prob(JB):                         0.00
Kurtosis:                       4.657   Cond. No.                         3.60
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [31]:
femodel = smf.mixedlm('routput ~ totlab +rcapstock', olsdata, groups=data['id'])
feresult = femodel.fit()
print feresult.summary()

          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: routput    
No. Observations: 20730   Method:             REML       
No. Groups:       2673    Scale:              0.1335     
Min. group size:  1       Likelihood:         -12895.3255
Max. group size:  18      Converged:          Yes        
Mean group size:  7.8                                    
----------------------------------------------------------
           Coef.  Std.Err.     z     P>|z|  [0.025  0.975]
----------------------------------------------------------
Intercept  7.116     0.045  157.791  0.000   7.027   7.204
totlab     0.616     0.007   85.294  0.000   0.602   0.630
rcapstock  0.194     0.004   43.859  0.000   0.186   0.203
Group Var  0.641     0.059                                



ImportError: No module named stats.plm